In [2]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List

import pandas as pd
import requests
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [3]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [4]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [5]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [6]:
# list_of_coins = pd.read_csv("/home/vlad/master_project/data/coin_list/list_of_coins.csv")['id'].to_list()

In [6]:
import boto3
session = boto3.Session(profile_name='master')

In [7]:
client = session.client('lambda')

In [15]:
type(client)

botocore.client.Lambda

In [11]:
client.list_functions()

{'ResponseMetadata': {'RequestId': 'e11613a1-8df9-4041-b78c-3fa7405e1116',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 20 Mar 2022 14:14:15 GMT',
   'content-type': 'application/json',
   'content-length': '992',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e11613a1-8df9-4041-b78c-3fa7405e1116'},
  'RetryAttempts': 0},
 'Functions': [{'FunctionName': 'Kosmo_Test_Lambda_Function',
   'FunctionArn': 'arn:aws:lambda:us-east-1:142327076576:function:Kosmo_Test_Lambda_Function',
   'Runtime': 'python3.9',
   'Role': 'arn:aws:iam::142327076576:role/Kosmobiker_Test_Lambda_Function_Role',
   'Handler': 'lambda_func.lambda_handler',
   'CodeSize': 1274851,
   'Description': '',
   'Timeout': 3,
   'MemorySize': 128,
   'LastModified': '2022-03-20T13:39:43.000+0000',
   'CodeSha256': 'patGhR2H6X6btZJQXsNDGXS6MdlY2wh+5VlWb6TkxgU=',
   'Version': '$LATEST',
   'TracingConfig': {'Mode': 'PassThrough'},
   'RevisionId': 'c67c9b47-b354-45f1-9b7f-9ee15b6902f4',
   'PackageType': 

In [12]:
def lambda_get_data(func_name: str, params: dict):
    response = client.invoke(
            FunctionName=func_name,
            InvocationType='Event',
            Payload=json.dumps(params)
        )
    return response

In [13]:
f = 'Kosmo_Test_Lambda_Function'
params = {
    "coin": "bitcoin",
    "bucket": "kosmobiker-masterproject",
    "path_raw_data": "data/raw"
}

In [14]:
lambda_get_data(f, params)

{'ResponseMetadata': {'RequestId': '979e0d1f-0f1f-45b9-99ec-ce84cd07cfbd',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 20 Mar 2022 14:20:11 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '979e0d1f-0f1f-45b9-99ec-ce84cd07cfbd',
   'x-amzn-remapped-content-length': '0',
   'x-amzn-trace-id': 'root=1-6237381b-278ecf32518bb425038bfd49;sampled=0'},
  'RetryAttempts': 0},
 'StatusCode': 202,
 'Payload': <botocore.response.StreamingBody at 0x7f2611bcc670>}